<a href="https://colab.research.google.com/github/arguntolga/ts_jupyter/blob/main/abovebelowma2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Define your two stock tickers
#stock_list = ["EREGL.IS", "FROTO.IS", "THYAO.IS", "TTRAK.IS", "MIATK.IS"]
stock_list = [
    "AKBNK.IS", "ARCLK.IS", "ASELS.IS", "BIMAS.IS", "BRISA.IS", "ASTOR.IS",
    "DOHOL.IS", "SASA.IS", "EKGYO.IS", "ENKAI.IS", "EREGL.IS", "FROTO.IS",
    "GARAN.IS", "GUBRF.IS", "HALKB.IS", "HEKTS.IS", "ISCTR.IS", "KCHOL.IS",
    "KOZAA.IS", "KOZAL.IS", "KRDMD.IS", "MGROS.IS", "TTRAK.IS", "PETKM.IS",
    #"PGSUS.IS", "SAHOL.IS", "SISE.IS", "TAVHL.IS", "THYAO.IS", "TKFEN.IS",
    "TTKOM.IS", "TUPRS.IS", "ULKER.IS", "VESTL.IS", "YKBNK.IS", "VESBE.IS"
]

# Set start and end dates
start_date = "2023-01-04"
end_date = "2024-05-30"

# Download data using yfinance
data = yf.download(stock_list, start=start_date, end=end_date)["Adj Close"]

#print(data)

def calculate_moving_averages(stock_list, data, window_sizes):
  for stock in stock_list:
    for window in window_sizes:
      data[stock + '_MA_' + str(window)] = data[stock].rolling(window=window).mean()
  return data

def calculate_above_below_ma(stock_list, data, window_sizes):
  above_below_counts = {}
  for date in data.index:
    daily_counts = {}
    for window in window_sizes:
      above_count = 0
      for stock in stock_list:
        ma_col = f"{stock}_MA_{window}"
        above_count += (data.loc[date, stock] > data.loc[date, ma_col])
      below_count = len(stock_list) - above_count
      daily_counts["Above MA" + str(window)] = above_count
      daily_counts["Below MA" + str(window)] = below_count
    above_below_counts[date] = daily_counts
  return pd.DataFrame(above_below_counts).transpose()


def calculate_super_stock(stock_list, data, window_sizes):
  super_stock = 0
  super_stock_df = pd.DataFrame(index=data.index, columns=stock_list)
  for date in data.index:
    for stock in stock_list:
      super_stock_level = 0
      if data.loc[date, stock] > data.loc[date, f"{stock}_MA_{window_sizes[0]}"]:
          super_stock_level += 1
      for window_idx in range(len(window_sizes) - 1):
        current_ma_col = f"{stock}_MA_{window_sizes[window_idx]}"
        next_ma_col = f"{stock}_MA_{window_sizes[window_idx + 1]}"
        if data.loc[date, current_ma_col] > data.loc[date, next_ma_col]:
          super_stock_level += 1
      super_stock_df.loc[date, stock] = int(super_stock_level)
  return super_stock_df


# Define window sizes for moving averages
window_sizes = [10, 20, 50, 100, 200]

# Apply the moving average calculation
data_with_ma = calculate_moving_averages(stock_list, data.copy(), window_sizes)
#display(data_with_ma)

# Calculate above/below MA counts
above_below_data = calculate_above_below_ma(stock_list, data_with_ma.copy(), window_sizes)
#print(above_below_data)

def green_red(value):
    if value < 15:
        return 'color: red; font-weight: bold'
    elif value == 0:
        return None
    else:
        return 'color: green; font-weight: bold'

above_below_data.iloc[::-1].style.applymap(green_red)



#display(above_below_data.iloc[::-1])

#super_stock_data = calculate_super_stock(stock_list, data_with_ma, window_sizes)
#cm = sns.light_palette("green", as_cmap=True)
#super_stock_data.iloc[::-1].apply(pd.to_numeric).style.set_sticky(axis=1).background_gradient(cmap=cm)

[*********************100%%**********************]  30 of 30 completed


,Above MA10,Below MA10,Above MA20,Below MA20,Above MA50,Below MA50,Above MA100,Below MA100,Above MA200,Below MA200
2024-05-29 00:00:00,8,22,16,14,22,8,24,6,25,5
2024-05-28 00:00:00,18,12,20,10,25,5,27,3,25,5
2024-05-27 00:00:00,19,11,20,10,26,4,27,3,25,5
2024-05-24 00:00:00,18,12,19,11,25,5,25,5,25,5
2024-05-23 00:00:00,22,8,24,6,26,4,26,4,25,5
2024-05-22 00:00:00,28,2,27,3,27,3,26,4,25,5
2024-05-21 00:00:00,24,6,26,4,26,4,26,4,25,5
2024-05-20 00:00:00,26,4,26,4,26,4,26,4,25,5
2024-05-17 00:00:00,21,9,25,5,23,7,24,6,24,6
2024-05-16 00:00:00,17,13,23,7,23,7,23,7,24,6
